In [228]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder

In [229]:
info_string = "age: continuous. workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. fnlwgt: continuous. education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. education-num: continuous. marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. sex: Female, Male. capital-gain: continuous. capital-loss: continuous. hours-per-week: continuous. native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands. income: <=50K, >50K"

In [230]:
info = pd.DataFrame(info_string.split('. '), columns=['detail'])
info['detail'] = info['detail'].apply(lambda x: x.split(': '))
# info['detail'] = info['detail'].apply(lambda x: [s.strip() for s in x])
info['col'] = info['detail'].apply(lambda x: x[0])
info['type'] = info['detail'].apply(lambda x: 'continuous' if x[1]=='continuous' else 'category')
info['category_label'] = info.apply(lambda x: x['detail'][1].split(', ') if x['type']=='category' else '', axis=1)
info['number_label'] = info['category_label'].apply(lambda x: len(x) if len(x)>0 else '')
info.drop(columns=['detail'], inplace=True)
info

,col,type,category_label,number_label
0,age,continuous,,
1,workclass,category,"[Private, Self-emp-not-inc, Self-emp-inc, Fede...",8
2,fnlwgt,continuous,,
3,education,category,"[Bachelors, Some-college, 11th, HS-grad, Prof-...",16
4,education-num,continuous,,
5,marital-status,category,"[Married-civ-spouse, Divorced, Never-married, ...",7
6,occupation,category,"[Tech-support, Craft-repair, Other-service, Sa...",14
7,relationship,category,"[Wife, Own-child, Husband, Not-in-family, Othe...",6
8,race,category,"[White, Asian-Pac-Islander, Amer-Indian-Eskimo...",5
9,sex,category,"[Female, Male]",2


In [231]:
train = pd.read_csv('data/adult/adult.data', header=None, names=info['col'], sep=', ')
train

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [232]:
# Create a function to evaluate missing in data, check unique in object (categorical col)
def check(df, info):
    check = []

    for col in df.columns:
        info_by_col = info[info['col']==col]
        type = df[col].dtypes
        type_info = info_by_col['type'].values[0]
        cate_label_info = info_by_col['category_label'].values[0]

        if type == 'object':
            unique_obj = df[col].unique()
        else:
            unique_obj = ''

        num_unique = len(unique_obj) if len(unique_obj) > 0 else ''
        num_cate_info = info_by_col['number_label'].values[0]

        diff = np.setdiff1d(list(unique_obj), list(cate_label_info))
        weird_obj = diff if len(diff) != 0 else ''

        total = df[col].isnull().sum()
        total_nan = total if total > 0 else ''

        percent = np.round(total/len(df[col])*100,2)
        percent_nan = percent if percent > 0 else ''

        check.append([col, type, type_info, total_nan, percent_nan, unique_obj, cate_label_info, num_unique, num_cate_info, weird_obj])

    check_df = pd.DataFrame(check, columns=['col', 'type', 'type_info', 'total_na', 'percent_na', 'unique_value', 'category_info', 'num_unique', 'num_cate_info', 'weird_obj'])
    return check_df
    
check(train, info)

,col,type,type_info,total_na,percent_na,unique_value,category_info,num_unique,num_cate_info,weird_obj
0,age,int64,continuous,,,,,,,
1,workclass,object,category,,,"[State-gov, Self-emp-not-inc, Private, Federal...","[Private, Self-emp-not-inc, Self-emp-inc, Fede...",9,8,[?]
2,fnlwgt,int64,continuous,,,,,,,
3,education,object,category,,,"[Bachelors, HS-grad, 11th, Masters, 9th, Some-...","[Bachelors, Some-college, 11th, HS-grad, Prof-...",16,16,
4,education-num,int64,continuous,,,,,,,
5,marital-status,object,category,,,"[Never-married, Married-civ-spouse, Divorced, ...","[Married-civ-spouse, Divorced, Never-married, ...",7,7,
6,occupation,object,category,,,"[Adm-clerical, Exec-managerial, Handlers-clean...","[Tech-support, Craft-repair, Other-service, Sa...",15,14,[?]
7,relationship,object,category,,,"[Not-in-family, Husband, Wife, Own-child, Unma...","[Wife, Own-child, Husband, Not-in-family, Othe...",6,6,
8,race,object,category,,,"[White, Black, Asian-Pac-Islander, Amer-Indian...","[White, Asian-Pac-Islander, Amer-Indian-Eskimo...",5,5,
9,sex,object,category,,,"[Male, Female]","[Female, Male]",2,2,


In [233]:
train_new = train.copy()

col_fillna = ['workclass', 'occupation', 'native-country']
cate_col = info[info['type'] == 'category']['col'].values.tolist()
num_col = info[info['type'] == 'continuous']['col'].values.tolist()
impute_col = train_new.drop(columns=['income']).columns.tolist()

ordinal_enc = OrdinalEncoder()
train_new[cate_col] = ordinal_enc.fit_transform(train_new[cate_col])

fill_back_na = SimpleImputer(missing_values=0, fill_value=np.nan, strategy='constant')
train_new[col_fillna] = fill_back_na.fit_transform(train_new[col_fillna])

std_scaler = StandardScaler()
train_new[num_col] = std_scaler.fit_transform(train_new[num_col])

knn_imputer = KNNImputer(n_neighbors=5)
train_new[impute_col] = knn_imputer.fit_transform(train_new[impute_col])
train_new

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0.030671,7.0,-1.063611,9.0,1.134739,4.0,1.0,1.0,4.0,1.0,0.148453,-0.21666,-0.035429,39.0,0.0
1,0.837109,6.0,-1.008707,9.0,1.134739,2.0,4.0,0.0,4.0,1.0,-0.145920,-0.21666,-2.222153,39.0,0.0
2,-0.042642,4.0,0.245079,11.0,-0.420060,0.0,6.0,1.0,4.0,1.0,-0.145920,-0.21666,-0.035429,39.0,0.0
3,1.057047,4.0,0.425801,1.0,-1.197459,2.0,6.0,0.0,2.0,1.0,-0.145920,-0.21666,-0.035429,39.0,0.0
4,-0.775768,4.0,1.408176,9.0,1.134739,2.0,10.0,5.0,2.0,0.0,-0.145920,-0.21666,-0.035429,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,-0.849080,4.0,0.639741,7.0,0.746039,2.0,13.0,5.0,4.0,0.0,-0.145920,-0.21666,-0.197409,39.0,0.0
32557,0.103983,4.0,-0.335433,11.0,-0.420060,2.0,7.0,0.0,4.0,1.0,-0.145920,-0.21666,-0.035429,39.0,1.0
32558,1.423610,4.0,-0.358777,11.0,-0.420060,6.0,1.0,4.0,4.0,0.0,-0.145920,-0.21666,-0.035429,39.0,0.0
32559,-1.215643,4.0,0.110960,11.0,-0.420060,4.0,1.0,3.0,4.0,1.0,-0.145920,-0.21666,-1.655225,39.0,0.0


In [234]:
train_final = train_new.copy()
train_final[num_col] = std_scaler.inverse_transform(train_final[num_col])
train_final[cate_col] = ordinal_enc.inverse_transform(train_final[cate_col])
train_final

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27.0,Private,257302.0,Assoc-acdm,12.0,Married-civ-spouse,Tech-support,Wife,White,Female,0.0,0.0,38.0,United-States,<=50K
32557,40.0,Private,154374.0,HS-grad,9.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
32558,58.0,Private,151910.0,HS-grad,9.0,Widowed,Adm-clerical,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K
32559,22.0,Private,201490.0,HS-grad,9.0,Never-married,Adm-clerical,Own-child,White,Male,0.0,0.0,20.0,United-States,<=50K


In [235]:
check(train_final, info)

,col,type,type_info,total_na,percent_na,unique_value,category_info,num_unique,num_cate_info,weird_obj
0,age,float64,continuous,,,,,,,
1,workclass,object,category,,,"[State-gov, Self-emp-not-inc, Private, Federal...","[Private, Self-emp-not-inc, Self-emp-inc, Fede...",8,8,
2,fnlwgt,float64,continuous,,,,,,,
3,education,object,category,,,"[Bachelors, HS-grad, 11th, Masters, 9th, Some-...","[Bachelors, Some-college, 11th, HS-grad, Prof-...",16,16,
4,education-num,float64,continuous,,,,,,,
5,marital-status,object,category,,,"[Never-married, Married-civ-spouse, Divorced, ...","[Married-civ-spouse, Divorced, Never-married, ...",7,7,
6,occupation,object,category,,,"[Adm-clerical, Exec-managerial, Handlers-clean...","[Tech-support, Craft-repair, Other-service, Sa...",14,14,
7,relationship,object,category,,,"[Not-in-family, Husband, Wife, Own-child, Unma...","[Wife, Own-child, Husband, Not-in-family, Othe...",6,6,
8,race,object,category,,,"[White, Black, Asian-Pac-Islander, Amer-Indian...","[White, Asian-Pac-Islander, Amer-Indian-Eskimo...",5,5,
9,sex,object,category,,,"[Male, Female]","[Female, Male]",2,2,
